1. Введение

In [12]:
from sympy import *
 
x, y = symbols('x, y')
eq1 = Eq(-4*x**3+18*x**2-8*x, 0)
 
sol = solve([eq1], [x])
sol


[(0,), (1/2,), (4,)]

In [13]:
-4**4+6*4**3-4*4**2 + 80

144

In [14]:
x, y, w = symbols('x, y, w')
eq1 = Eq(2*x-w, 0)
eq2 = Eq(4*y-w, 0)
eq3 = Eq(x+y-20, 0)
 
sol = solve([eq1, eq2, eq3], [x,y,w])
13**2+2*7**2


267

2. Градиентный спуск: применение и модификации

In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn import linear_model
df = sns.load_dataset('diamonds')

In [16]:
df.drop(['depth', 'table', 'x', 'y', 'z'], axis=1, inplace=True)

In [17]:
df = pd.get_dummies(df, drop_first=True)

In [18]:
df['carat'] = np.log(1+df['carat'])
df['price'] = np.log(1+df['price'])

In [19]:
X = df.drop(columns="price")
y = df["price"]

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42,  test_size = 0.33)

In [21]:
param_grid = {"loss": ["squared_error", "epsilon_insensitive"],
              "penalty": ["elasticnet"],
              "alpha": np.logspace(-3, 3, 10),
              "l1_ratio": np.linspace(0, 1, 10),
              "learning_rate": ["constant"],
              "eta0": np.logspace(-4, -1, 4)
              }
grid_search = GridSearchCV(
    estimator=linear_model.SGDRegressor(
        random_state=42,
        # max_iter=1000
    ), 
    param_grid=param_grid, 
    # cv=5, 
    n_jobs = -1
)  
%time grid_search.fit(X_train, y_train) 

y_test_pred = grid_search.predict(X_test)
print(f'MSE на тестовом наборе: {(root_mean_squared_error(y_test, y_test_pred))**2:.3f}')
print(f"Наилучшие значения гиперпараметров: {grid_search.best_params_}")

sgd = linear_model.SGDRegressor(**grid_search.best_params_, random_state = 42)

sgd.fit(X_train, y_train)
sgd.score(X_train, y_train) # r2
ls = sgd.predict(X_test)

round(root_mean_squared_error(y_test, ls), 3)

CPU times: total: 42 s
Wall time: 1min 52s
MSE на тестовом наборе: 0.044
Наилучшие значения гиперпараметров: {'alpha': 0.001, 'eta0': 0.001, 'l1_ratio': 0.0, 'learning_rate': 'constant', 'loss': 'epsilon_insensitive', 'penalty': 'elasticnet'}


0.209